In [49]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import folium
from folium import plugins

In [50]:
df = pd.read_csv('populated_bird_data.csv')

m = folium.Map(location=[20, 15], zoom_start=3, tiles='CartoDB dark_matter')

In [51]:
def get_smooth_path(df_subset):
    if df_subset.empty:
        return []
    df_subset = df_subset.sort_values('day_of_year')
    df_subset['week'] = df_subset['day_of_year'] // 7
    weekly = df_subset.groupby('week')[['latitude', 'longitude']].mean().reset_index()
    return list(zip(weekly['latitude'], weekly['longitude']))

In [ ]:
species_list = df['species'].unique()
print(f"Processing {len(species_list)} species...")

for bird_name in species_list:
    print(f"  - Training model for: {bird_name}")

    data = df[df['species'] == bird_name].copy()
    
    data['date'] = pd.to_datetime(data[['year', 'month', 'day']])
    data['day_of_year'] = data['date'].dt.dayofyear
    
    X = data[['year', 'day_of_year']]
    y = data[['latitude', 'longitude']]
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X, y)
    
    future_days = np.arange(1, 366)
    X_future = pd.DataFrame({'year': [2026]*365, 'day_of_year': future_days})
    pred_coords = model.predict(X_future)
    
    future_data = X_future.copy()
    future_data['latitude'] = pred_coords[:, 0]
    future_data['longitude'] = pred_coords[:, 1]
    
    path_2016 = get_smooth_path(data[data['year'] == 2016].copy())
    path_2024 = get_smooth_path(data[data['year'] == 2024].copy())
    path_2026 = get_smooth_path(future_data.copy())
    
    fg = folium.FeatureGroup(name=bird_name)
    
    if path_2016:
        folium.PolyLine(
            path_2016, color='#00FFFF', weight=2, opacity=0.6, 
            tooltip=f'{bird_name} 2016'
        ).add_to(fg)
        
    if path_2024:
        folium.PolyLine(
            path_2024, color='#FF00FF', weight=3, opacity=0.8, 
            tooltip=f'{bird_name} 2024'
        ).add_to(fg)
        
    if path_2026:
        folium.PolyLine(
            path_2026, color='#32CD32', weight=3, opacity=0.9, 
            dash_array='5, 10', tooltip=f'{bird_name} 2026 (Pred)'
        ).add_to(fg)
        folium.CircleMarker(
            path_2026[0], radius=3, color='#32CD32', fill=True, 
            popup=f'{bird_name} 2026 Start'
        ).add_to(fg)

    fg.add_to(m)

Processing 10 species...
  - Training model for: Blackcap
  - Training model for: Brambling
  - Training model for: Collared flycatcher
  - Training model for: Common Redstart
  - Training model for: Eurasian Siskin
  - Training model for: European pied flycatcher
  - Training model for: Spotted flycatcher
  - Training model for: Willow warbler
  - Training model for: Wood Pigeon
  - Training model for: Wood warbler


In [53]:
legend_html = '''
     <div style="position: fixed; 
     bottom: 50px; left: 50px; width: 150px; height: 120px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color:rgba(255, 255, 255, 0.8);
     padding: 10px; border-radius: 5px;">
     <b>Migration Legend</b><br>
     <i style="color:#00FFFF">___</i> 2016 (Actual)<br>
     <i style="color:#FF00FF">___</i> 2024 (Actual)<br>
     <i style="color:#32CD32">_ _</i> 2026 (Predicted)<br>
     <br>
     <small>*Toggle species in top-right</small>
     </div>
     '''
m.get_root().html.add_child(folium.Element(legend_html))

In [54]:
m.save("all_birds_migration_prediction.html")
print("Map generated: all_birds_migration_prediction.html")

Map generated: all_birds_migration_prediction.html
